In [18]:
!pip3 install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4 pypdf langchain_text_splitters sentence_transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [1]:
import os
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

loader = PyPDFLoader("../rag/tmp/docs/Truc Nguyen CV.pdf")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = splitter.split_documents(documents)

print(docs)

embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    docs, embedding, persist_directory=os.path.join(os.curdir, "vectordb")
)
vectorstore.persist()

[Document(page_content='Truc\nNguyen\nFull\nStack\n.NET\nDeveloper\n(+44)751\n489\n7979\n-\ntruchjkl@gmail.com\nOverview\nI\nam\na\nfull\nstack\n.NET\ndeveloper\nwith\nmore\nthan\n9\nyears\nof\nexperience\nin \n.NET\ntechnologies.\nI\nhave\nmainly\nfocused\non\ndeveloping\nscalable\nand \nwell-documented\ncode.\nI\nenjoy\nworking\ncollaboratively\nbut\nalso\ncan\nrun\nthe \nproject\nindependently.\nTechnical\nSkills\n●\nProgramming\nLanguages\n○\nC#\n/\nVB.NET,\nJavaScript/TypeScript,\nDart\n○\nGolang\n(side\nproject\nat\nhttps://github.com/truc9/goal\n)\n●\nBackend\n○\n.NET\nCore,\nASP.NET\nCore,\nASP.NET\nMVC\n/\nWeb\nAPI,\nBackground \nWorker\n○\nSignalR,\nMediatR\n○\nUnit\nTest\nwith\nMoq\n●\nFrontEnd\n○\nHTML,\nCSS\n○\njQuery\n○\nBootstrap,\nTailwindCSS,\nDevExtreme,\nTelerik\nKendo\nUI\n○\nAngularJS,\nAngular\n●\nMobile:\nFlutter\n●\nDatabase:\nMicrosoft\nSQL\nServer\n&\nEntity\nFramework\nCore\n●\nVersion\ncontrol:\nGIT\n●\nAWS\nMSK\n(Kafka)\n●\nAWS\nLambda\nFunction\nWork\nExpe

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

llm = ChatOllama(model="llama3")
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma(
    persist_directory=os.path.join(os.curdir, "vectordb"),
    embedding_function=embedding,
)
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

question = "Tell me about Truc?"

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = rag_chain.invoke(question)
print(answer)

Based on the provided context, Truc is a full stack .NET developer with over 9 years of experience in .NET technologies.
